# **Modelo de Machine Learning**

*A continuación se muestran los resultados del modelo de entrenamiento para un sistema de recomendación basado en **item-item** que identifica que tan similar es un item con respecto al resto y con base a ello realiza un **conjunto de recomendaciones**.*

**Nota:** *Esta función se creará en el archivo [main](main.py)*

#### **Importación de librerías** ####
---

*Para este cuaderno usaremos las siguientes librerías: **pandas**, **Scikit Learn** y **Pyarrow**.*

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
dfGames= pd.read_parquet("dataout/out_games.parquet")
dfGenres = pd.read_parquet("dataout/out_genres_games.parquet")

#### **Función recomendacionJuego** ####
---

*La función **recomendacionJuego** le permite al usuario **filtrar** por un **id de aplicación** específico y mostrarle una lista de recomendaciones de **5 apps similares** a esa aplicación buscada.*

#### Creamos el dataframe que contiene los indices de aplicación, los ids de aplicación y sus nombres para facilitar esta función ####

*Este dataframe solo contará con las columnas que nos interesan:*

1. *IdApp.*
2. *Name.*


In [3]:
dfApps = dfGames[["IdApp", "Name"]]

#### Creamos el dataframe dfSimilitudes ####

*Este dataframe será la matriz de similitudes que facilitará la busqueda de aplicaciones con mayor similitud.*

1. *Las columnas y los indices serán, los indices correspondientes a cada idApp en la columna **dfApps**.*


In [4]:
dfSimilitudes = pd.DataFrame(cosine_similarity(dfGenres.iloc[:,1:]))
dfSimilitudes.head(5)

,0,1,2,3,4,5,6,7,8,9,...,22519,22520,22521,22522,22523,22524,22525,22526,22527,22528
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.707107,0.0,0.0,1.0,1.0,0.0,1.0,0.707107,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.707107,0.0,0.0,1.0,1.0,0.0,1.0,0.707107,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.707107,0.0,0.0,1.0,1.0,0.0,1.0,0.707107,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.707107,0.0,0.0,1.0,1.0,0.0,1.0,0.707107,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.707107,0.0,0.0,1.0,1.0,0.0,1.0,0.707107,1.0


#### Creamos la función recomedacionJuego ####

*Esta función recibirá como parametro:*

1. *dev = Cadena string con el nombre del desarrollador.*


In [5]:
dfApps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22529 entries, 0 to 22528
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   IdApp   22529 non-null  int64 
 1   Name    22529 non-null  object
dtypes: int64(1), object(1)
memory usage: 352.1+ KB


In [6]:
def recomendacionJuego(idApp):
    # Primero convertimos el input en un valor numérico para facilitar la busqueda.
    idApp = int(idApp)
    
    # Si el valor no se encuentra retornar que no se encontró.
    if idApp not in dfApps["IdApp"]:
        return "No se encuentra el id ingresado dentro de la base de datos."
    
    # Ahora vamos a buscar el indice que corresponde al IdApp
    indSearch = dfApps.index[dfApps["IdApp"] == idApp][0]
    
    # Ahora vamos a identificar la fila en el dataframe de similitudes usando el indice
    filaApp = dfSimilitudes.iloc[indSearch]
    
    # Ahora eliminamos el valor de la fila que corresponde al indice que estabamos buscando, ya que no necesitamos obtener la similitud de la misma app.
    filaApp.drop(index=indSearch, inplace=True)
    
    # Ordenamos los valores de esta fila en orden descendente para tomar los índices de los 5 elementos más similares.
    result = filaApp.sort_values(ascending=False).index[1:6]
    
    appsMasSim = dfGames.loc[result, 'Name']
    
    return appsMasSim

In [7]:
recomendacionJuego(20)

5053                                          MechRunner
4989          Call of Duty®: Ghosts - Classic Ghost Pack
4990    Call of Duty®: Ghosts - Drill Instructor VO Pack
4991          Call of Duty®: Ghosts - Snoop Dogg VO Pack
4993                                    Metro 2033 Redux
Name: Name, dtype: object

*Revisamos que funcione correctamente:*

#### Guardamos el dataframe para las consultas de la API ####

*Para efectos de este ejercicio no guardaremos el dataframe por almacenamiento.*

In [33]:
#simTab = pa.Table.from_pandas(dfSimilitudes)
#dir = "datafunc/similitudes.parquet"
#pq.write_table(simTab,dir)